In [ ]:
"""
형태소 분류 라이브러리를 사용하여 단어를 분리
단어들을 분류한 집합을 Bag of words라고 한다.
단어들을 품사별로 분류하는 라이브러리는 매우 많다.
한글의 경우 koNLPy(꼬꼬마, Twitter-> Okt, Komoran, Hannanum, Mecab, Open Korean Text)
영어의 경우 NLTK를 주로 사용한다.
어떤 라이브러리를 사용할지는 텍스트 마이닝의 목적에 따라서 선택한다.

원본 데이터가 정상적인 경우는 형태소가 잘 분류가 되는데, 정상적이지 않은 경우는 분류가 잘 안된다.
단어의 중요도는 어떻게 평가할까?
여러 문서에서 자주 나오는 단어같은 경우 중요도가 낮다고 생각하고 전처리 작업으로 제거해준다.
TF는 기본적인 던어 중요도를 평가하는 스코어이다. 모든 단어의 빈도수에 대한 특정 단어의 빈도수
빈도수는 문서에 비례하여 증가하는 경향으로 중요도를 평가하는 좋은 척도라고 볼 수 없다.
IDF 역문헌 빈도를 도입하게 되는데, 전체 문서에서 특정 단어가 포함된 문서가 차지하는 비율의 역수로 표현
즉 log(해당 단어가 포함된 문서의 수에 대한 전체 문서의 수)가 된다.
--> 특정 단어가 나오는 문서의 수가 많은 수록 높은값이 나온다.
이는 텍스트 마이닝에서 자주 쓰이는 단어 중요 척도이다.
"""

In [52]:
# codecs 모듈은 임의의 코덱으로 인코딩과 디코딩하는 함수을 지원
# pip install koNLPy
# java를 설치해야 사용할 수 있다. 라이브러리가 java를 사용하기 때문!
import codecs
from konlpy.tag import Okt

twitter = Okt()
line = '나는 국민대학교의 축제에 놀러 가고 싶습니다.'
malist = twitter.pos(line)
malist

[('나', 'Noun'),
 ('는', 'Josa'),
 ('국민', 'Noun'),
 ('대학교', 'Noun'),
 ('의', 'Josa'),
 ('축제', 'Noun'),
 ('에', 'Josa'),
 ('놀러', 'Verb'),
 ('가고', 'Verb'),
 ('싶습니다', 'Verb'),
 ('.', 'Punctuation')]

In [53]:
from konlpy.tag import Komoran
tagger = Komoran()

line = '나는 국민대학교의 축제에 놀러 가고 싶습니다.'
malist = tagger.pos(line)
malist

[('나', 'NP'),
 ('는', 'JX'),
 ('국민대학교', 'NNP'),
 ('의', 'JKG'),
 ('축제', 'NNG'),
 ('에', 'JKB'),
 ('놀', 'VV'),
 ('러', 'EC'),
 ('가', 'VV'),
 ('고', 'EC'),
 ('싶', 'VX'),
 ('습니다', 'EF'),
 ('.', 'SF')]

In [54]:
from konlpy.tag import Komoran

tagger = Komoran()  # 형태소 분석기
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import requests
import lxml.html
import codecs

#투캅스 시나리오 읽어오기
#이 시나리오 데이터는 제공되어질수 없으며 배포가 불가능하오니 
#http://www.korean.go.kr(국립국어원)에서 회원가입을 하시고 다운을 받아주세요. 
articles = []
fp = codecs.open("투캅스.txt", "r", encoding="utf-8")
#soup = BeautifulSoup(fp, "html.parser")
#body = soup.select_one("body") # html파일을 분석할 수 있어서 body부분만 추출가능
#text = body.getText()
#articles = text.split("\n")
#len(articles)
articles = fp.read()
articles = articles.split("\n")
list = []
for i in articles:
    if i:
        list.append(i.strip())
fp.close()


In [55]:
articles = list
articles

['[투캅스]',
 'S#1 프롤로그 (밤)',
 '1. (화면 가득히 보이는 도로 양 옆의 현란한',
 '아크릴 간판들)',
 '2. (화려한 거리)',
 '3. (패트롤 카, 밤거리를 달려 나간다.)',
 '4. (자정 열두시가 되는 시계)',
 '5. (간판 불이 하나 둘 탁탁 꺼진다.)',
 '6. (꺼지는 간판들)',
 '7. (가라오께의 간판불이 꺼진다.)',
 '8.(웨이터 하나가 또다른 셔터에 자물쇠를 채우고',
 '지하 계단을 내려간다. 웨이터가 사라지는 계단',
 '아래쪽은 완전 적막!)',
 '9. (좌르르! 내려 닫히는 출입구의 육중한 셔터',
 '도어)',
 '10. (채워지는 철문)',
 '11. (패트롤이 지나치면 차 안에서 망보던 부엉이가',
 '몸을 일으킨다.)',
 '부엉이: 여긴 부엉이!',
 '12. 사장: 수고했다. 올빼미는 어떤가?',
 '13. 올빼미: 상황 끝.',
 '14. 사장: 좋아, 작전 개시!',
 '15. (배전판을 올리는 손)',
 '16. (불이 켜지는 샹드리에)',
 '17. 디제이: 네, 오랫동안 기다리셨습니다. 3번',
 '테이블 손님 권영락 씨 다함께 차차차!',
 '(마이크를 잡고 목이 터져라 뽕짝을 불러 제끼는',
 '사내. 멀티비젼이 장치된 화려한 무대 위에서 춤추는',
 '사람들)',
 '18. (복도를 따라 보이는 각 룸들의 흥겨운 분위기)',
 '19. (웨이터 하나가 계산서가 놓인 쟁반을 들고',
 '문을 열고 들어온다. 계산서를 집어보는 뒷모습의 조',
 '형사)',
 '조 형사: 이봐, 김 전무! 자네 한달 봉급이 얼마지?',
 '김 형사: 56만 원!',
 '조 형사: 젠장, 한 달 봉급을 두 시간 만에',
 '날렸구만! 카드도 되지요?',
 '웨이터: 아, 예. 물론입니다.',
 '20. 조 형사: (지갑 꺼내 보이며) 저 비자',
 '카든데...',
 '웨이터: 아! 됩니다.',
 '조 형사: 가만 있자 카드가 어디 갔지?',
 '21. (기가 막힌 표정의 웨이터)',
 '2

In [57]:
# 텍스트를 처리하기
results = []
lines = articles
words_all = []

for line in lines:
    # 형태소 분석하기
    malist = twitter.pos(line, norm=True, stem=True)
    print(malist)
    
    r = []
    for word in malist:
        # 명사/동사/부사만 걸러내기  [0]:단어, [1]:품사
        if word[1] in ['Noun','Verb','Adjective']:
            r.append(word[0])
            words_all.append(word[0])
    rl = (" ".join(r)).strip()
    results.append(rl)
    #print(rl)

[('[', 'Punctuation'), ('투캅스', 'Noun'), (']', 'Punctuation')]
[('S', 'Alpha'), ('#', 'Punctuation'), ('1', 'Number'), ('프롤로그', 'Noun'), ('(', 'Punctuation'), ('밤', 'Noun'), (')', 'Punctuation')]
[('1', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('화면', 'Noun'), ('가득하다', 'Adjective'), ('보이다', 'Verb'), ('도로', 'Noun'), ('양', 'Noun'), ('옆', 'Noun'), ('의', 'Josa'), ('현란하다', 'Adjective')]
[('아크릴', 'Noun'), ('간판', 'Noun'), ('들', 'Suffix'), (')', 'Punctuation')]
[('2', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('화려하다', 'Adjective'), ('거리', 'Noun'), (')', 'Punctuation')]
[('3', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('패', 'Noun'), ('트롤', 'Noun'), ('카', 'Noun'), (',', 'Punctuation'), ('밤', 'Noun'), ('거리', 'Noun'), ('를', 'Josa'), ('달다', 'Verb'), ('나가다', 'Verb'), ('.)', 'Punctuation')]
[('4', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('자정', 'Noun'), ('열두시', 'Noun'), ('가', 'Josa'), ('되다', 'Verb'), ('시계', 'Noun'), (')', 'Punctuation')]
[('5', 'Num

[('부엉이', 'Noun'), (':', 'Punctuation'), ('주방', 'Noun'), ('쪽', 'Noun'), ('에요', 'Josa'), ('.', 'Punctuation')]
[('(', 'Punctuation'), ('조', 'Noun'), ('형사', 'Noun'), ('와', 'Josa'), ('김', 'Noun'), ('형사', 'Noun'), ('오다', 'Verb'), ('길', 'Noun'), ('로', 'Josa'), ('죽다', 'Verb'), ('내', 'Noun'), ('빼다', 'Verb'), ('.)', 'Punctuation')]
[('28', 'Number'), ('.(', 'Punctuation'), ('자물쇠', 'Noun'), ('에', 'Josa'), ('불꽃', 'Noun'), ('이', 'Josa'), ('튀다', 'Verb'), ('.)', 'Punctuation')]
[('29', 'Number'), ('.(', 'Punctuation'), ('쾅', 'Noun'), ('!', 'Punctuation'), ('문', 'Noun'), ('을', 'Josa'), ('밀', 'Noun'), ('치고', 'Josa'), ('들어오다', 'Verb'), ('조', 'Noun'), ('형사', 'Noun'), ('와', 'Josa'), ('김', 'Noun'), ('형사', 'Noun'), ('.', 'Punctuation')]
[('그', 'Noun'), ('바람', 'Noun'), ('에', 'Josa'), ('문', 'Noun'), ('을', 'Josa'), ('열리다', 'Verb'), ('웨이터', 'Noun'), ('가', 'Josa'), ('뒤', 'Noun'), ('로', 'Josa'), ('벌렁', 'Noun')]
[('자빠지다', 'Verb'), ('.)', 'Punctuation')]
[('30', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'

[('흘낏거리', 'Noun'), ('며', 'Josa'), ('신참', 'Noun'), ('녀석', 'Noun'), ('의', 'Josa'), ('기색', 'Noun'), ('을', 'Josa'), ('살피', 'Noun'), ('며', 'Josa'), (')', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('경찰', 'Noun'), ('학교', 'Noun'), ('를', 'Josa'), ('수석', 'Noun'), ('졸업', 'Noun'), ('하다', 'Verb'), ('하다', 'Verb'), ('?', 'Punctuation')]
[('강', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('(', 'Punctuation'), ('가슴', 'Noun'), ('을', 'Josa'), ('쭉', 'Noun'), ('펴다', 'Verb'), (')', 'Punctuation'), ('네', 'Noun'), (',', 'Punctuation'), ('그렇다', 'Adjective'), ('.', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('뭐', 'Noun'), ('먹다', 'Verb'), ('있다', 'Adjective'), ('경찰', 'Noun'), ('이', 'Josa'), ('돼다', 'Verb'), ('?', 'Punctuation')]
[('강', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('네', 'Noun'), ('?', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('추다', 'Verb'), ('배고프다', 'Adjective'), ('직업', 'Noun'), ('이야', 'Josa'), ('!', 'Punctuati

[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('얼마', 'Noun'), ('지', 'Josa'), ('?', 'Punctuation')]
[('카운터', 'Noun'), (':', 'Punctuation'), ('11만', 'Number'), ('2천원', 'Number'), ('입니다', 'Foreign'), ('.', 'Punctuation')]
[('박', 'Noun'), ('사장', 'Noun'), (':', 'Punctuation'), ('어허', 'Exclamation'), ('이', 'Determiner'), ('거', 'Noun'), ('왜', 'Noun'), ('이', 'Determiner'), ('러시', 'Noun'), ('나', 'Josa'), (',', 'Punctuation'), ('우리', 'Noun'), ('동네', 'Noun'), ('까지', 'Josa')]
[('오다', 'Verb'), ('.', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('아', 'Exclamation'), ('이르다', 'Verb'), ('멀다', 'Adjective'), ('안되다', 'Adjective'), ('.', 'Punctuation'), ('한두', 'Modifier'), ('번', 'Noun'), ('도', 'Josa'), ('아니다', 'Adjective'), ('.', 'Punctuation')]
[('박', 'Noun'), ('사장', 'Noun'), (':', 'Punctuation'), ('어허', 'Exclamation'), ('다시', 'Noun'), ('안', 'Noun'), ('볼', 'Noun'), ('사이', 'Noun'), ('도', 'Josa'), ('아니다', 'Adjective'), ('이르다', 'Verb')]
[('쓰다', 'Verb'), ('.', 'Punctuation')]


[('7', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('달려오다', 'Verb'), ('차', 'Noun'), ('에', 'Josa'), ('부딪히다', 'Verb'), ('소매치기', 'Noun'), (')', 'Punctuation')]
[('8', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('땅', 'Noun'), ('으로', 'Josa'), ('구르다', 'Verb'), ('떨어지다', 'Verb'), ('소매치기', 'Noun'), ('와', 'Josa'), ('그', 'Noun'), ('뒤', 'Noun'), ('로', 'Josa'), ('따르다', 'Verb')]
[('강', 'Noun'), ('형사', 'Noun'), (')', 'Punctuation')]
[('S', 'Alpha'), ('#', 'Punctuation'), ('19', 'Number'), ('형사', 'Noun'), ('과', 'Josa')]
[('1', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('마치', 'Noun'), ('시', 'Modifier'), ('장통', 'Noun'), ('처럼', 'Josa'), ('시끌벅적하다', 'Adjective'), ('형사', 'Noun'), ('과', 'Josa'), ('안', 'Noun'), ('의', 'Josa'), ('풍경', 'Noun'), ('.', 'Punctuation')]
[('그', 'Noun'), ('한쪽', 'Noun'), ('에', 'Josa'), ('소매치기', 'Noun'), ('와', 'Josa'), ('책상', 'Noun'), ('에', 'Josa'), ('마주', 'Noun'), ('앉다', 'Verb'), ('조서', 'Noun'), ('를', 'Josa'), ('꾸미다', 'Verb')]
[('있다', 'Adjective'), (

...

[('여', 'Noun'), ('주인', 'Noun'), (':', 'Punctuation'), ('아', 'Exclamation'), ('.', 'Punctuation'), ('네', 'Noun'), ('에', 'Josa'), ('---', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('저', 'Noun'), ('그렇다', 'Adjective'), ('!', 'Punctuation')]
[('S', 'Alpha'), ('#', 'Punctuation'), ('92', 'Number'), ('대', 'Modifier'), ('저택', 'Noun'), ('앞', 'Noun')]
[('1', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('대', 'Modifier'), ('저택', 'Noun'), ('앞', 'Noun'), ('의', 'Josa'), ('투캅스', 'Noun'), ('차', 'Noun'), ('.', 'Punctuation'), ('조', 'Noun'), ('형사', 'Noun'), ('오르다', 'Verb'), ('타다', 'Verb'), (')', 'Punctuation')]
[('2', 'Number'), ('.', 'Punctuation'), ('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('젠장', 'Noun'), ('생기다', 'Verb'), ('거', 'Noun'), ('없이', 'Adverb'), ('골치', 'Noun'), ('만', 'Josa'), ('아프다', 'Adjective')]
[('생기다', 'Verb'), ('.', 'Punctuation'), ('가지', 'Noun'), ('.', 'Punctuation'), ('안', 'Noun'), ('가다', 'Verb'), ('?', 'Punctuation')]
[('강', 'Noun'), ('

[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('야', 'Exclamation'), (',', 'Punctuation'), ('임마', 'Noun'), ('.', 'Punctuation'), ('야', 'Exclamation'), ('이', 'Noun'), ('새끼', 'Noun'), ('야', 'Josa'), ('!', 'Punctuation')]
[('S', 'Alpha'), ('#', 'Punctuation'), ('102', 'Number'), ('어느', 'Adverb'), ('아파트', 'Noun')]
[('1', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('퍽', 'Noun'), ('!', 'Punctuation'), ('소리', 'Noun'), ('와', 'Josa'), ('함께', 'Adverb'), ('침대', 'Noun'), ('위로', 'Noun'), ('벌렁', 'Noun'), ('나자빠지다', 'Verb')]
[('여자', 'Noun'), ('하나', 'Noun'), (')', 'Punctuation')]
[('2', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('다가오다', 'Verb'), ('잔혹하다', 'Adjective'), ('인상', 'Noun'), ('의', 'Josa'), ('사내', 'Noun'), (')', 'Punctuation')]
[('3', 'Number'), ('.(', 'Punctuation'), ('신음', 'Noun'), ('을', 'Josa'), ('토', 'Noun'), ('하고', 'Josa'), ('있다', 'Adjective'), ('여자', 'Noun'), ('를', 'Josa'), ('일으키다', 'Verb'), ('세우다', 'Verb'), ('...)', 'Punctuation')]
[('태', 'Noun'), ('성', 'S

[('황조', 'Noun'), (':', 'Punctuation'), ('그렇다', 'Adjective'), ('보다', 'Verb'), ('야구', 'Noun'), ('방망이', 'Noun'), ('아니다', 'Adjective'), ('사시미', 'Noun'), ('겠다', 'Verb'), ('.', 'Punctuation')]
[('아마', 'Noun'), ('그거', 'Noun'), ('한', 'Determiner'), ('방이', 'Noun'), ('면', 'Josa'), ('다', 'Adverb'), ('도망가다', 'Verb'), ('걸', 'Noun'), ('요', 'Josa'), ('?', 'Punctuation')]
[('3', 'Number'), ('.', 'Punctuation'), ('태', 'Noun'), ('성', 'Suffix'), (':', 'Punctuation'), ('그거', 'Noun'), ('?', 'Punctuation'), ('푸다', 'Verb'), ('하하', 'Noun'), ('하', 'Suffix'), ('....', 'Punctuation'), ('맞다', 'Verb'), ('맞다', 'Verb'), ('!', 'Punctuation')]
[('(', 'Punctuation'), ('자신', 'Noun'), ('만만하다', 'Adjective'), ('보이다', 'Verb'), ('둘', 'Noun'), (')', 'Punctuation')]
[('S', 'Alpha'), ('#', 'Punctuation'), ('113', 'Number'), ('폐공', 'Noun'), ('장', 'Suffix')]
[('1', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('멀리', 'Noun'), ('서', 'Josa'), ('달려오다', 'Verb'), ('그랜저', 'Noun'), ('정면', 'Noun'), ('으로', 'Josa'), ('오다', 'Verb')

[('62', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('일어나다', 'Verb'), ('강', 'Noun'), ('형사', 'Noun'), ('의', 'Josa'), ('뒷모습', 'Noun'), ('.', 'Punctuation'), ('졸개', 'Noun'), ('가', 'Josa'), ('총', 'Noun'), ('을', 'Josa')]
[('겨누', 'Noun'), ('고', 'Josa'), ('있다', 'Adjective'), (')', 'Punctuation')]
[('63', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('강', 'Noun'), ('형사', 'Noun'), ('의', 'Josa'), ('일어나다', 'Verb'), ('앞', 'Noun'), ('모습', 'Noun'), (')', 'Punctuation')]
[('64', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('한발', 'Adverb'), ('걸어오다', 'Verb'), ('총', 'Noun'), ('겨누', 'Noun'), ('는', 'Josa'), ('졸개', 'Noun'), (')', 'Punctuation')]
[('65', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('총', 'Noun'), ('쏘다', 'Verb'), ('폼', 'Noun'), ('을', 'Josa'), ('잡다', 'Verb'), ('강', 'Noun'), ('형사', 'Noun'), (')', 'Punctuation')]
[('66', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('총', 'Noun'), ('쏘다', 'Verb'), ('졸개', 'Noun'), (')', 'Punctuation')]
[('67', 'N

[('112', 'Number'), ('.', 'Punctuation'), ('(', 'Punctuation'), ('차', 'Noun'), ('에', 'Josa'), ('오르다', 'Verb'), ('투캅스', 'Noun'), ('.', 'Punctuation'), ('조', 'Noun'), ('형사', 'Noun'), (',', 'Punctuation'), ('안', 'Noun'), ('주머니', 'Noun'), ('에서', 'Josa')]
[('돈', 'Noun'), ('뭉치', 'Noun'), ('두', 'Noun'), ('개', 'Noun'), ('를', 'Josa'), ('꺼내다', 'Verb'), ('.)', 'Punctuation')]
[('강', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('이', 'Noun'), ('게', 'Josa'), ('뭡', 'Noun'), ('니까', 'Josa'), ('?', 'Punctuation')]
[('조', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('수입', 'Noun'), ('이', 'Josa'), ('생기', 'Noun'), ('면', 'Josa'), ('반', 'Noun'), ('으로', 'Josa'), ('나누다', 'Verb'), ('게', 'Noun'), ('네', 'Noun')]
[('법', 'Noun'), ('이', 'Determiner'), ('라며', 'Noun'), ('?', 'Punctuation'), ('싫다', 'Adjective'), ('내', 'Noun'), ('가', 'Josa'), ('갖다', 'Verb'), ('!', 'Punctuation')]
[('강', 'Noun'), ('형사', 'Noun'), (':', 'Punctuation'), ('(', 'Punctuation'), ('돈', 'Noun'), ('을', 'Josa'), ('챙기다', 'Verb'), (')', 'Punctua

In [58]:
words_all

['투캅스',
 '프롤로그',
 '밤',
 '화면',
 '가득하다',
 '보이다',
 '도로',
 '양',
 '옆',
 '현란하다',
 '아크릴',
 '간판',
 '화려하다',
 '거리',
 '패',
 '트롤',
 '카',
 '밤',
 '거리',
 '달다',
 '나가다',
 '자정',
 '열두시',
 '되다',
 '시계',
 '간판',
 '불',
 '하나',
 '둘',
 '탁탁',
 '꺼지다',
 '꺼지다',
 '간판',
 '가르다',
 '간판',
 '불',
 '꺼지다',
 '웨이터',
 '하나',
 '셔터',
 '자물쇠',
 '채우다',
 '지하',
 '계단',
 '내려가다',
 '웨이터',
 '사라지다',
 '계단',
 '아래쪽',
 '완전',
 '적막',
 '좌르르',
 '내다',
 '닫히다',
 '출입구',
 '육중',
 '셔터',
 '도어',
 '채우다',
 '철문',
 '패',
 '트롤',
 '지나치다',
 '차',
 '안',
 '망',
 '보다',
 '부엉이',
 '몸',
 '일으키다',
 '부엉이',
 '여기다',
 '부엉이',
 '사장',
 '수고',
 '하다',
 '올빼미',
 '어떻다',
 '올빼미',
 '상황',
 '끝',
 '사장',
 '좋다',
 '작전',
 '개시',
 '배',
 '전판',
 '올리다',
 '손',
 '불',
 '켜지다',
 '샹드리',
 '디제이',
 '네',
 '기다리다',
 '번',
 '테이블',
 '손님',
 '권',
 '영락',
 '씨',
 '차차차',
 '마이크',
 '잡고',
 '목',
 '터지다',
 '뽕짝',
 '불다',
 '끼',
 '사내',
 '멀티비젼',
 '장치',
 '되다',
 '화려하다',
 '무대',
 '위',
 '춤추다',
 '사람',
 '복도',
 '따르다',
 '보이다',
 '각',
 '룸들',
 '흥겹다',
 '분위기',
 '웨이터',
 '하나',
 '계산서',
 '놓이다',
 '쟁반',
 '들다',
 '문',
 '열다',
 '들어오다',
 '계산서',
 '집다',
 '보다',
 '